In [64]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [ ]:
user_feature_embedding = pd.read_csv('data/user_feature_embedding.csv')

In [115]:
def read_data(files, batch_size):  
    COLUMN_DEFAULTS = [[''], [''], [''], [''], [''], [''], [''], [''], [''], [''], [''], [''], ['']]
    COLUMNS = ['interest1', 'interest2', 'interest3', 'interest4', 'interest5',
               'kw1', 'kw2', 'kw3',
               'topic1', 'topic2', 'topic3',
               'appIdInstall', 'appIdAction']
    
    def convert(value):
        splits = tf.string_split([value], '-')
        dense = tf.sparse_tensor_to_dense(splits, default_value='0')
        return tf.string_to_number(dense, out_type=tf.int32)
    
    def parser(value):
        interest1, interest2, interest3, interest4, interest5, \
               kw1, kw2, kw3, \
               topic1, topic2, topic3, \
               appIdInstall, appIdAction = tf.decode_csv(value, COLUMN_DEFAULTS)
        return convert(interest1), convert(interest2), convert(interest3), convert(interest4), convert(interest5),\
            convert(kw1), convert(kw2), convert(kw3), \
            convert(topic1), convert(topic2), convert(topic3), \
            convert(appIdInstall), convert(appIdAction)
        
    dataset = tf.data.TextLineDataset(files).skip(1).map(parser).batch(batch_size)
    text_line = dataset.make_one_shot_iterator().get_next()
    
    return text_line

In [121]:
tf.reset_default_graph()
with tf.get_default_graph().as_default():
    interest1, interest2, interest3, interest4, interest5, \
               kw1, kw2, kw3, \
               topic1, topic2, topic3, \
               appIdInstall, appIdAction = read_data('data/user_feature_embedding.csv', 1)


In [123]:
# just test 
with tf.Session() as sess:
    for i in range(2):
        # interest1, interest2, interest3, interest4, interest5, \
        #        kw1, kw2, kw3, \
        #        topic1, topic2, topic3, \
        #        appIdInstall, appIdAction = sess.run([interest1, interest2, interest3, interest4, interest5, \
        #        kw1, kw2, kw3, \
        #        topic1, topic2, topic3, \
        #        appIdInstall, appIdAction])
        int1, int2 = sess.run([interest1, interest2])
        print(int1)
        print(int2)
        print()


[[[ 93  70  77  86 109  47  75  69  45   8  29  49  83   6  46  36  11
    44  30 118  76  48  28 106  59  67  41 114 111  71   9]]]
[[[46 19 13 29]]]

[[[75 29]]]
[[[33]]]



In [125]:
with tf.get_default_graph().as_default():
    with tf.name_scope('embedding'):
        embedding = tf.Variable(tf.random_uniform([123, 1], -1, 1))
        interest1_embedded = tf.nn.embedding_lookup(embedding, interest1)
        interest1_embedded_sum = tf.reduce_sum(interest1_embedded, -2)